# Source: Physician Compare

## Read Raw Data

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

path = 'Data/DAC_NationalDownloadableFile.csv'
RawData = pd.read_csv(path, low_memory = False, encoding = 'ISO-8859-1')
RawData.columns = [i.strip() for i in RawData.columns]
cols = ['NPI','frst_nm',  'mid_nm', 'lst_nm', 'suff', 'gndr', 'Cred', 'pri_spec', 'cty', 'st', 'zip']

RawData = RawData[cols]
# RawData = RawData.drop_duplicates().reset_index(drop = True)
RawData

In [ ]:
# NPI and its accounts
print('Unique NPI we have: {}'.format(len(RawData['NPI'].value_counts())))
RawData['NPI'].value_counts()

In [ ]:
# RawData['Cred'].value_counts()#.sum()
UniqueRawData = RawData.groupby('NPI').first().reset_index()
UniqueRawData

In [ ]:
UniqueRawData.fillna('Missing')['Cred'].value_counts()

## Generate Keywords

In [ ]:
def get_doctor_info(DoctorDF):
    doctor_infolist = []
    for _, row in DoctorDF.iterrows():
        # most results: fistname is before lastname
        if row['mid_nm'] == "NaN":
            doctor_info = "{} {} {} {} {}".format(row['frst_nm'], row['lst_nm'], 
                                                  row['pri_spec'], 
                                                  row['cty'], row['st'],
                                                  # row['zip']
                                                    )
        else:
            doctor_info = "{} {} {} {} {} {}".format(row['frst_nm'], row['mid_nm'],  row['lst_nm'], 
                                                  row['pri_spec'], 
                                                  row['cty'], row['st'],
                                                  # row['zip']
                                                )
        # print(doctor_info)
        doctor_infolist.append([row['NPI'], doctor_info])

    return doctor_infolist

## MDData

In [ ]:
RawData2 = RawData[RawData['Cred'] == 'MD'].reset_index(drop = True)
MDData = RawData2.groupby('NPI').first().reset_index()
MDData = MDData.fillna('NaN')
MDData.to_pickle('Output/_MDData.p')
MDData['keyword'] = get_doctor_info(MDData)
MDData[['NPI', 'keyword']].to_pickle('Output/MDDataKeywords.p')
print(MDData.shape)

## MissingData

In [ ]:
RawData2 = RawData[RawData['Cred'].isna()].reset_index(drop = True)
MissingData = RawData2.groupby('NPI').first().reset_index()
MissingData = MissingData.fillna('NaN')
MissingData.to_pickle('Output/_MissingData.p')
MissingData['keyword'] = get_doctor_info(MissingData)
MissingData[['NPI', 'keyword']].to_pickle('Output/MissingDataKeywords.p')
print(MissingData.shape)

## Split Half and Half

In [ ]:
old_md_doc_dir = 'Output/MD1_DocListDF_All.p'

Old_MD_Doc = pd.read_pickle(old_md_doc_dir)
existed_npi = Old_MD_Doc.iloc[:100000]['NPI'].unique()
len(existed_npi)

In [ ]:
MD2_Data = MDData[-MDData['NPI'].isin(existed_npi)].reset_index(drop = True)
MD2_Data

In [ ]:
Second_MDData[['NPI', 'keyword']].to_pickle('Output/Second_MDDataKeywords.p')

# Source: NPPES Data

## Read Raw Data

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)


cols = ['NPI',
 'Entity Type Code',
 'Replacement NPI',
 'Employer Identification Number (EIN)',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider Other Organization Name',
 'Provider Other Organization Name Type Code',
 'Provider Other Last Name',
 'Provider Other First Name',
 'Provider Other Middle Name',
 'Provider Other Name Prefix Text',
 'Provider Other Name Suffix Text',
 'Provider Other Credential Text',
 'Provider Other Last Name Type Code',
 'Provider First Line Business Mailing Address',
 'Provider Second Line Business Mailing Address',
 'Provider Business Mailing Address City Name',
 'Provider Business Mailing Address State Name',
 'Provider Business Mailing Address Postal Code',
 'Provider Business Mailing Address Country Code (If outside U.S.)',
 'Provider Business Mailing Address Telephone Number',
 'Provider Business Mailing Address Fax Number',
 'Provider First Line Business Practice Location Address',
 'Provider Second Line Business Practice Location Address',
 'Provider Business Practice Location Address City Name',
 'Provider Business Practice Location Address State Name',
 'Provider Business Practice Location Address Postal Code',
 'Provider Business Practice Location Address Country Code (If outside U.S.)',
 'Provider Business Practice Location Address Telephone Number',
 'Provider Business Practice Location Address Fax Number',
 'Provider Enumeration Date',
 'Last Update Date',
 'NPI Deactivation Reason Code',
 'NPI Deactivation Date',
 'NPI Reactivation Date',
 'Provider Gender Code',
 'Authorized Official Last Name',
 'Authorized Official First Name',
 'Authorized Official Middle Name',
 'Authorized Official Title or Position',
 'Authorized Official Telephone Number',
 'Healthcare Provider Taxonomy Code_1',
 'Provider License Number_1',
 'Provider License Number State Code_1',
 'Healthcare Provider Primary Taxonomy Switch_1',
 'Healthcare Provider Taxonomy Code_2',
 'Provider License Number_2',
 'Provider License Number State Code_2',
 'Healthcare Provider Primary Taxonomy Switch_2',
 'Healthcare Provider Taxonomy Code_3',
 'Provider License Number_3',
 'Provider License Number State Code_3',
 'Healthcare Provider Primary Taxonomy Switch_3',
 'Healthcare Provider Taxonomy Code_4',
 
 'Other Provider Identifier_1',
 'Other Provider Identifier Type Code_1',
 'Other Provider Identifier State_1',
 'Other Provider Identifier Issuer_1',
 'Other Provider Identifier_2',
 'Other Provider Identifier Type Code_2',
 'Other Provider Identifier State_2',
 'Other Provider Identifier Issuer_2',
 'Other Provider Identifier_3',
 'Other Provider Identifier Type Code_3',
 'Other Provider Identifier State_3',
 'Other Provider Identifier Issuer_3',

 
 'Is Sole Proprietor',
 'Is Organization Subpart',
 'Parent Organization LBN',
 'Parent Organization TIN',
 'Authorized Official Name Prefix Text',
 'Authorized Official Name Suffix Text',
 'Authorized Official Credential Text',
 'Healthcare Provider Taxonomy Group_1',
 'Healthcare Provider Taxonomy Group_2',
 'Healthcare Provider Taxonomy Group_3',
 'Healthcare Provider Taxonomy Group_4',
 'Healthcare Provider Taxonomy Group_5',
 'Healthcare Provider Taxonomy Group_6',
 'Healthcare Provider Taxonomy Group_7',
 'Healthcare Provider Taxonomy Group_8',
 'Healthcare Provider Taxonomy Group_9',
 'Healthcare Provider Taxonomy Group_10',
 'Healthcare Provider Taxonomy Group_11',
 'Healthcare Provider Taxonomy Group_12',
 'Healthcare Provider Taxonomy Group_13',
 'Healthcare Provider Taxonomy Group_14',
 'Healthcare Provider Taxonomy Group_15',
 'Certification Date',]


def check_MD_or_DO(x):
    if type(x) == str:
        x = x.replace(' ', '').replace('.', '').replace(';', '').split(',')
        return 'MD' in x or 'DO' in x # or 'MD,' in x or '.MD' 
    else:
        return False

In [ ]:
import pandas as pd

filename = 'data/npidata_pfile_20050523-20210808.csv'
chunksize = 10000


D = {}
idx = 1


total = 0
indtotal = 0
indus_total = 0
indus_md_ob_total = 0


L = []
for chunk in pd.read_csv(filename, chunksize=chunksize, low_memory=False):
    break
    
chunk

In [ ]:
list(chunk.columns)

In [ ]:
import pandas as pd

filename = 'data/npidata_pfile_20050523-20210808.csv'
chunksize = 10000


D = {}
idx = 1


total = 0
indtotal = 0
indus_total = 0
indus_md_ob_total = 0


L = []
for chunk in pd.read_csv(filename, chunksize=chunksize, low_memory=False):
    df = chunk[cols]
    df2 = df[df['Entity Type Code'] == 1.]
    df3 = df2[df2['Provider Business Practice Location Address Country Code (If outside U.S.)'] == 'US']
    df4 = df3[df3['Provider Credential Text'].apply(check_MD_or_DO)]
    L.append(df4)
    print(idx, len(df), len(df2), len(df3), len(df4))
    total += len(df)
    indtotal += len(df2)
    indus_total += len(df3)
    indus_md_ob_total += len(df4)
    idx += 1
    

In [ ]:
print('Total {}, Ind {}, Ind US {}'.format(total, indtotal, indus_md_ob_total))

In [ ]:
IndUS_MD_OD_DF = pd.concat(L).reset_index(drop = True)
IndUS_MD_OD_DF

In [ ]:
def change_cred(x):
    if type(x) == str:
        x = x.replace(' ', '').replace('.', '').replace(';', '')
        x = x[:-1] if x[-1] == ',' else x
        return x
    else:
        return None

IndUS_MD_OD_DF['Cred'] = IndUS_MD_OD_DF['Provider Credential Text'].apply(change_cred)
# IndUS_MD_OD_DF.to_pickle('Output/IndUS_MD_OD_DF.p')
IndUS_MD_OD_DF

In [ ]:
IndUS_MD_OD_DF['Cred'].value_counts()

## Taxomony Code

In [ ]:
import pandas as pd
path = 'Data/nucc_taxonomy_211.csv'

TaxoCode = pd.read_csv(path)
TaxoCode

In [ ]:
# https://nucc.org/index.php/code-sets-mainmenu-41/provider-taxonomy-mainmenu-40/csv-mainmenu-57

col = 'Code'
keys = list(TaxoCode[col].values)
print(len(keys))

col = 'Display Name'
d = TaxoCode[col].fillna('Missing')# .apply(lambda x: x.split('/')[-1].split(', ')[-1].replace('\x92s', '').strip())
values = list(d.values)
print(len(values))


Code2Speciality = dict(zip(keys, values))
Code2Speciality

In [ ]:
IndUS_MD_OD_DF[col].value_counts()# = IndUS_MD_OD_DF[col].apply(lambda x: Code2Speciality[x])

In [ ]:
# list(d.values)
# col = 'PROVIDER TAXONOMY DESCRIPTION: TYPE, CLASSIFICATION, SPECIALIZATION'
col = 'Healthcare Provider Taxonomy Code_1'
IndUS_MD_OD_DF['Speciality'] = IndUS_MD_OD_DF[col].apply(lambda x: Code2Speciality.get(x, 'Missing'))

In [ ]:
IndUS_MD_OD_DF['Speciality'].value_counts()

In [ ]:
IndUS_MD_OD_DF.to_pickle('Output/IndUS_MD_OD_DF.p')

## Remain NPPES

In [ ]:
medicare_npis = list(UniqueRawData[UniqueRawData['Cred'].fillna('Missing').isin(['MD', 'Missing'])]['NPI'].values)
print(len(medicare_npis))

In [ ]:
df = IndUS_MD_OD_DF
IndUS_MD_OD_DF2 = df[-df['NPI'].isin(medicare_npis)].reset_index(drop = True)
IndUS_MD_OD_DF2.shape

In [ ]:
IndUS_MD_OD_DF2

In [ ]:
def get_doctor_info_v2(DoctorDF):
    doctor_infolist = []
    for _, row in DoctorDF.iterrows():
        # most results: fistname is before lastname
        
        if pd.isnull(row['Provider Middle Name']):
            doctor_info = "{} {} {} {} {}".format(row['Provider First Name'], row['Provider Last Name (Legal Name)'], 
                                                  row['Speciality'], 
                                                  row['Provider Business Mailing Address City Name'], row['Provider Business Mailing Address State Name'],
                                                  # row['zip']
                                                    )
        else:
            doctor_info = "{} {} {} {} {} {}".format(row['Provider First Name'], row['Provider Middle Name'],  row['Provider Last Name (Legal Name)'], 
                                                  row['Speciality'], 
                                                  row['Provider Business Mailing Address City Name'], row['Provider Business Mailing Address State Name'],
                                                  # row['zip']
                                                )
        # print(doctor_info)
        doctor_infolist.append([row['NPI'], doctor_info])

    return doctor_infolist

In [ ]:
IndUS_MD_OD_DF2['keyword'] = get_doctor_info_v2(IndUS_MD_OD_DF2)
IndUS_MD_OD_DF2[['NPI', 'keyword']].to_pickle('Output/IndUS2_MD_OD_DF_Keywords.p')